# **NOTEBOOK D'UTILISATION DE CERTAINES FONCTIONALITEES PERSO**

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
from datetime import datetime
from collections import Counter
from geoalchemy2 import Geometry,WKTElement
import geopandas as gp
import pandas as pd
import numpy as np
import shutil

from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom
from Decorateurs import concat_df

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 50)

In [ ]:
#TELECHARGER DES DONNEES depuis leserveur geomatique et les passer dans un fichier
@concat_df
def creer_dept(fichier):
    return gp.read_file(fichier)

liste_f=[fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP' for d in  ['16','17','86','79','19','23','87','33','24','40','64','47']]
df_tot=creer_dept(liste_f)

In [7]:
#trsnfere des données dans une bdd
with ct.ConnexionBdd('local_otv_station_gti') as c :
    for d in ['17','86','79','19','23','87','33','24','40','64','47'] :
        fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP'
        ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='plamade', table='troncon_route_bdt17_l',SRID='2154',
                          geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8')

debut import fichier D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_017.SHP avec shape2pg à 22:39:44 
 avec commande ogr2ogr -append -update -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt MULTILINESTRINGZ -dim 3 -lco "SCHEMA=plamade" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432" D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_017.SHP -nln plamade.troncon_route_bdt17_l 
Fait
debut import fichier D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_086.SHP avec shape2pg à 22:40:04 
 avec commande ogr2ogr -append -update -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt MULTILINESTRINGZ -dim 3 -lco "SCHEMA=plamade" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432" D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_086.SHP -nln plamade.troncon_route_bdt17_l 
Fait
debut import fichier D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_079.SHP avec shape2pg à 22:40:21 

In [6]:
with ct.ConnexionBdd('local_otv_station_gti') as c :
    fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_016.SHP'
    ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='plamade', table='troncon_route_bdt17_l',SRID='2154',
                          geotype='MULTILINESTRINGZ', dims=3, creationMode='',encodageClient='UTF-8')

debut import fichier D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_016.SHP avec shape2pg à 22:39:15 
 avec commande ogr2ogr  -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt MULTILINESTRINGZ -dim 3 -lco "SCHEMA=plamade" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432" D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_016.SHP -nln plamade.troncon_route_bdt17_l 
Fait
